In [1]:
# -*- coding: utf-8 -*- 

import re 
from tqdm import tqdm 
import time
from datetime import datetime
import sqlite3
import sys
import os
import pandas as pd
import unify

In [2]:
def connect(file_path, primary, columns):
    con = sqlite3.connect(file_path)
    cur = con.cursor()
    cols = ", ".join([c + ' Varchar' for c in columns]) 
    cur.execute("create table meta ("+primary+" Varchar PRIMARY KEY, "+cols+" )")
    cur.execute("CREATE INDEX log on meta (id);")
    cur.execute("create table plain_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL);")
    cur.execute("create table tagged_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL );")
    con.commit()
    return con, cur

In [3]:
workdir = r'D:\Arzamas'
filename = 'Arzamas.db'
file_path = os.path.join(workdir, filename)
metatablepath = os.path.join(workdir,'metatable.csv')
tagged = os.path.join(workdir,'tagged_texts')
plain = os.path.join(workdir,'texts')

meta = pd.read_csv(metatablepath, sep='\t', encoding='utf8')
meta.head()

,source,id,title,theme,author,textdate,tags,URL,Author_profession,About_author
0,Arzamas,400,Бессмысленный тест на знание Arzamas •,[],NaN,30.01.2017,[],http://arzamas.academy/mag/400-two-years,NaN,NaN
1,Arzamas,299,Черновики Михаила Булгакова •,['Литература'],Мария Котова,14.06.2016,[],http://arzamas.academy/mag/299-bulgakov,Историк литературы,"Кандидат филологических наук, ведущий научный ..."
2,Arzamas,164,За что на самом деле сожгли Джордано Бруно? •,['История'],Ованес Акопян,05.11.2015,[],http://arzamas.academy/mag/164-bruno,Историк,"Кандидат исторических наук, специалист по исто..."
3,Arzamas,282,5 новых книг о литературе •,['Литература'],NaN,05.04.2016,[],http://arzamas.academy/mag/282-essay,NaN,NaN
4,Arzamas,303,Зачем американцы красили лицо в черный •,"['Искусство', 'Антропология']",Феликс Сандалов,04.07.2016,[],http://arzamas.academy/mag/303-menestrels,Журналист,Автор книги «Формейшен: история одной сцены»


In [4]:

if not os.path.exists(filename):
    con, cur = connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
else:
    con = sqlite3.connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
    cur = con.cursor()

In [5]:

cur.execute("SELECT * FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('table', 'meta', 'meta', 2, 'CREATE TABLE meta (id Varchar PRIMARY KEY, source Varchar, title Varchar, theme Varchar, author Varchar, textdate Varchar, tags Varchar, URL Varchar, Author_profession Varchar, About_author Varchar )'), ('table', 'plain_texts', 'plain_texts', 5, 'CREATE TABLE plain_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL)'), ('table', 'tagged_texts', 'tagged_texts', 7, 'CREATE TABLE tagged_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL )')]


In [16]:
meta.iloc[6].to_dict()

{'About_author': 'Кандидат искусствоведения, доцент Санкт-Петербургского государственного университета промышленных технологий и\xa0дизайна, лектор Русского музея, автор книг «Всадники особого назначения» (2013), «Костюм и\xa0мода Российской империи. Эпоха Николая\xa0II» (2013), «Костюм и\xa0мода Российской империи. Эпоха Александра\xa0II и\xa0Александра\xa0III» (2015), «Молодые и\xa0красивые. Мода 1920-х годов» (2016).',
 'Author_profession': 'Искусствовед, историк моды',
 'URL': 'http://arzamas.academy/mag/208-russo-turkish-war',
 'author': 'Ольга Хорошилова',
 'id': 208,
 'source': 'Arzamas',
 'tags': '[]',
 'textdate': '07.12.2015',
 'theme': "['История', 'Антропология']",
 'title': 'Как Русско-турецкая война повлияла на моду  •  '}

In [9]:
for i in range(len(meta)):
    values = meta.iloc[i].to_dict()
    values['id'] = str(values['id'])
    columns = ', '.join(values.keys())
    print(columns)
    print(values)
    placeholders = ', '.join('?' * len(values))
    sql = 'INSERT INTO meta ({}) VALUES ({})'.format(columns, placeholders)
    cur.execute(sql, values.values())
    
    valuest = {'id': values['id'], 'text': unify.open_text(os.path.join(plain, str(values['id'])+".txt"))}
    columns = ', '.join(valuest.keys())
    placeholders = ', '.join('?' * len(valuest))
    sql2 = 'INSERT INTO plain_texts ({}) VALUES ({})'.format(columns, placeholders) 
    cur.execute(sql2, valuest.values())
    
    valuest = {'id': values['id'], 'text': unify.open_text(open(os.path.join(tagged, str(values['id'])+".txt"),'r', encoding='utf8').read())}
    columns = ', '.join(valuest.keys())
    placeholders = ', '.join('?' * len(valuest))
    sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
    cur.execute(sql3, valuest.values())
    
    con.commit()

source, id, title, theme, author, textdate, tags, URL, Author_profession, About_author
{'source': 'Arzamas', 'id': '400', 'title': 'Бессмысленный тест на знание Arzamas  •  ', 'theme': '[]', 'author': nan, 'textdate': '30.01.2017', 'tags': '[]', 'URL': 'http://arzamas.academy/mag/400-two-years', 'Author_profession': nan, 'About_author': nan}


ValueError: parameters are of unsupported type